In [1]:
# Download yolov8n-pose backbone (cached by Ultralytics)
from pathlib import Path
from ultralytics import YOLO

target = Path('yolov8n-pose.pt')
if target.exists():
    print('yolov8n-pose.pt already exists at', target)
else:
    print('Downloading yolov8n-pose.pt (this will be cached by Ultralytics) ...')
    # Loading via YOLO(...) will download and cache the weights if not present
    _ = YOLO('yolov8n-pose.pt')
    print('Download triggered. The file will be available from Ultralytics cache; if you need a local copy, you can export or copy it from the cache.')


Download triggered. The file will be available from Ultralytics cache; if you need a local copy, you can export or copy it from the cache.
Download triggered. The file will be available from Ultralytics cache; if you need a local copy, you can export or copy it from the cache.


In [ ]:
# Train (local-friendly version)
import os, yaml
from pathlib import Path
import torch
from ultralytics import YOLO

# Detect Kaggle vs local
IS_KAGGLE = bool(os.environ.get('KAGGLE_URL_BASE')) or Path('/kaggle/working').exists()
print('Environment:', 'Kaggle' if IS_KAGGLE else 'Local')

# Dataset preference order:
# 1. merged_dataset/data_merged.yaml (nếu đã merge)
# 2. dataset4corner/dataset.yaml
# 3. dataset/data.yaml
candidates = [
    Path('merged_dataset/data_merged.yaml'),
    Path('dataset4corner/dataset.yaml'),
    Path('dataset/data.yaml'),
]
source_yaml = next((p for p in candidates if p.exists()), None)
assert source_yaml is not None, 'Không tìm thấy bất kỳ YAML dataset nào (merged_dataset/data_merged.yaml | dataset4corner/dataset.yaml | dataset/data.yaml)'
print('Using source dataset YAML ->', source_yaml)

with open(source_yaml, 'r') as f:
    data = yaml.safe_load(f) or {}

# --- PATH NORMALIZATION + FALLBACKS ---
# Một số YAML (vd dataset4corner/dataset.yaml) có thể chứa đường dẫn tuyệt đối từ máy khác -> gây lỗi.
raw_path = data.get('path')
print('Raw path from YAML:', raw_path)

# Ưu tiên: nếu người dùng set FORCE_DATASET_PATH thì dùng luôn
force_path = os.environ.get('FORCE_DATASET_PATH')
if force_path:
    print('FORCE_DATASET_PATH override ->', force_path)
    data['path'] = force_path

# Nếu không có override, xử lý logic mặc định
if not force_path:
    # base ứng viên lấy từ YAML nếu có, ngược lại folder chứa YAML
    base_candidate = Path(raw_path) if raw_path else source_yaml.parent

    # Nếu base_candidate là absolute nhưng không tồn tại -> fallback
    if raw_path and not base_candidate.exists():
        print('[WARN] Absolute path trong YAML không tồn tại -> fallback source folder')
        base_candidate = source_yaml.parent

    # Nếu thư mục ảnh train không tồn tại trong base_candidate -> thử các fallback khác
    expected_train = base_candidate / 'images' / 'train'
    expected_val   = base_candidate / 'images' / 'val'
    if not expected_train.exists() or not expected_val.exists():
        print('[WARN] images/train hoặc images/val không tồn tại dưới base_candidate:', base_candidate)
        # Thử fallback 1: chính thư mục bên cạnh YAML (source_yaml.parent)
        alt1 = source_yaml.parent
        if (alt1 / 'images' / 'train').exists():
            print(' -> Fallback thành công: dùng', alt1)
            base_candidate = alt1
        else:
            # Thử fallback 2: không đặt path (YOLO sẽ đọc đường dẫn tương đối)
            print(' -> Fallback alt1 thất bại. Xoá khóa path để YOLO dùng relative paths.')
            data.pop('path', None)
            base_candidate = None

    if base_candidate is not None:
        data['path'] = str(base_candidate.resolve())

# Nếu vẫn chưa có path key, bảo đảm train/val là relative
if 'path' not in data:
    print('[INFO] Sử dụng relative paths (không có khóa path trong YAML output).')

# Chuẩn hoá train/val key (nếu YAML cũ dùng image/train ... vẫn giữ nguyên)
train_key = data.get('train', 'images/train')
val_key   = data.get('val', 'images/val')

# Loại bỏ test nếu không rõ
if not data.get('test'):
    data.pop('test', None)

# Đảm bảo các khóa pose
data.setdefault('names', ['plate'])
data.setdefault('kpt_shape', [4, 3])
data.setdefault('skeleton', [[0,1],[1,2],[2,3],[3,0]])
data.setdefault('flip_idx', [0,1,2,3])

# Ghi lại train/val (phòng trường hợp thay đổi biến)
data['train'] = train_key
data['val']   = val_key

# Đường dẫn YAML xuất ra
if IS_KAGGLE:
    out_yaml = Path('/kaggle/working/data.yaml')
else:
    out_yaml = Path('data_local.yaml')  # local file trong project root

out_yaml.parent.mkdir(parents=True, exist_ok=True)
with open(out_yaml, 'w') as f:
    yaml.safe_dump(data, f, sort_keys=False)
print('Written normalized YAML ->', out_yaml)
print('--- Final YAML content ---')
print(yaml.safe_dump(data, sort_keys=False))

# Sanity check tồn tại thư mục
paths_to_check = []
base_for_check = Path(data['path']) if 'path' in data else source_yaml.parent
paths_to_check.extend([
    base_for_check/'images'/'train',
    base_for_check/'images'/'val',
    base_for_check/'labels'/'train',
    base_for_check/'labels'/'val',
])
for p in paths_to_check:
    print('Exists:', p, p.exists())

missing_crit = [p for p in paths_to_check[:2] if not p.exists()]
if missing_crit:
    raise FileNotFoundError(f"❌ Không tìm thấy thư mục ảnh train/val sau khi chuẩn hoá: {missing_crit}. Hãy kiểm tra lại dataset4corner cấu trúc hoặc chỉnh FORCE_DATASET_PATH.")

# Chọn checkpoint
ckpt_candidates = [
    Path('runs/pose_plate_merged/weights/best.pt'),
    Path('runs/pose_plate/weights/best.pt'),
    Path('best.pt'),
]
ckpt = next((p for p in ckpt_candidates if p.exists()), None)
if ckpt:
    print(f'Loading checkpoint: {ckpt}')
    model = YOLO(str(ckpt))
else:
    print('No existing checkpoint found -> init new small pose model')
    backbone_opts = ['yolo11n-pose.pt','yolov8n-pose.pt']
    backbone = next((b for b in backbone_opts if Path(b).exists()), None)
    model = YOLO(backbone) if backbone else YOLO('yolov8n-pose.pt')  # sẽ tải nếu có mạng

# Tham số train nhanh để TEST local
EPOCHS = int(os.environ.get('TEST_EPOCHS', '5'))  # đổi biến môi trường nếu muốn
BATCH  = int(os.environ.get('TEST_BATCH',  '4'))

train_args = dict(
    data=str(out_yaml),
    imgsz=640,
    epochs=EPOCHS,
    patience=0,              # Không early stop trong test ngắn
    batch=BATCH,
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=2 if IS_KAGGLE else 4,
    project='runs',
    name='pose_plate_local_test',
    exist_ok=True,
    pretrained=False,        # tránh tải online
    cache='ram',
    plots=True,
    save_period=0,
    seed=42,
    amp=False,               # bạn có thể bật True nếu torch + GPU hỗ trợ
    verbose=True,
)
print('Train args:', train_args)

results = model.train(**train_args)
print('Done. Best metrics path in runs/pose_plate_local_test')